In [2]:
import cv2
import numpy as np
import os
import sys
from PIL import Image
from glob import glob
from tqdm import tqdm   
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)
        

In [11]:
folder_list=glob('../../data/stroke_CT/**/**/')
for i in tqdm(range(len(folder_list))):
    folder=folder_list[i]

    create_dir(folder.replace('stroke_CT','stroke_CT_preprocessed'))
    img_list=glob(folder+'*.png')
    for j in range(len(img_list)):
        img_path=img_list[j]
        img=Image.open(img_path)
        img=img.resize((512,512))
        img.save(img_path.replace('stroke_CT','stroke_CT_preprocessed'))

        


100%|██████████| 2435/2435 [33:54<00:00,  1.20it/s]


In [5]:
folder_list=glob('../../data/Ischemic_stroke_mri/**/**/**/')
for i in tqdm(range(len(folder_list))):
    folder=folder_list[i]

    create_dir(folder.replace('Ischemic_stroke_mri','stroke_MRI_preprocessed'))
    img_list=glob(folder+'*.png')
    for j in range(len(img_list)):
        img_path=img_list[j]
        img=Image.open(img_path)
        img=img.resize((256,256))
        img.save(img_path.replace('Ischemic_stroke_mri','stroke_MRI_preprocessed'))


100%|██████████| 5300/5300 [36:15<00:00,  2.44it/s]  


In [4]:
folder_list

['../../data/Ischemic_stroke_mri/image/ADC/ANO1_1195/',
 '../../data/Ischemic_stroke_mri/image/ADC/ANO1_1196/',
 '../../data/Ischemic_stroke_mri/image/ADC/ANO1_1197/',
 '../../data/Ischemic_stroke_mri/image/ADC/ANO1_1198/',
 '../../data/Ischemic_stroke_mri/image/ADC/ANO1_1199/',
 '../../data/Ischemic_stroke_mri/image/ADC/ANO1_1200/',
 '../../data/Ischemic_stroke_mri/image/ADC/ANO1_1201/',
 '../../data/Ischemic_stroke_mri/image/ADC/ANO1_1202/',
 '../../data/Ischemic_stroke_mri/image/ADC/ANO1_1203/',
 '../../data/Ischemic_stroke_mri/image/ADC/ANO1_1204/',
 '../../data/Ischemic_stroke_mri/image/ADC/ANO1_1205/',
 '../../data/Ischemic_stroke_mri/image/ADC/ANO1_1206/',
 '../../data/Ischemic_stroke_mri/image/ADC/ANO1_1207/',
 '../../data/Ischemic_stroke_mri/image/ADC/ANO1_1208/',
 '../../data/Ischemic_stroke_mri/image/ADC/ANO1_1209/',
 '../../data/Ischemic_stroke_mri/image/ADC/ANO1_1210/',
 '../../data/Ischemic_stroke_mri/image/ADC/ANO1_1211/',
 '../../data/Ischemic_stroke_mri/image/ADC/ANO1_